In [1]:
import numpy as np 
import open3d as o3d 
import matplotlib.pyplot as plt
from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [67]:
o3d.core.cuda.is_available()

False

In [110]:
input_path = Path('../data/fountain-P11')
data_name = input_path / 'sparse.ply'
pcd = o3d.io.read_point_cloud(str(data_name))

In [72]:
front = [ -0.11383137417328129, -0.52838800964009403, -0.8413373458502853 ]
lookat = [ -0.0839009317646284, -0.075120607621831828, 0.24466620489292801 ]
up = [ 0.072004008076520493, -0.84900242887201671, 0.52345993026241544 ]
zoom = 0.059999999999999998

In [111]:
pcd_center = pcd.get_center()
print(pcd_center)
pcd.translate(-pcd_center)

[ 0.01815032 -0.49427251  5.4572703 ]


PointCloud with 11721 points.

In [112]:
print(pcd.get_center())

[ 2.57413747e-14 -1.37572555e-15 -7.62601312e-15]


### Statistical outlier filter

In [113]:
nn = 16 # nearest neighbours 
std_multiplier = 10 # it is is above that, it is considered outlier

filtered_pcd = pcd.remove_statistical_outlier(nn, std_multiplier)

In [114]:
outliers = pcd.select_by_index(filtered_pcd[1], invert=True)

In [115]:
outliers.paint_uniform_color([0,1,0])
pcd_ww_outliers = filtered_pcd[0]

o3d.visualization.draw_geometries([pcd_ww_outliers, outliers], zoom=zoom, front=front, lookat=lookat, up=up)

### Voxel downsampling

In [105]:
voxel_size = 0.01 # 10 cm 

pcd_downsampled = pcd_ww_outliers.voxel_down_sample(voxel_size)
o3d.visualization.draw_geometries([pcd_downsampled], zoom=zoom, front=front, lookat=lookat, up=up)

### Estimating normals

In [116]:
nn_distance = np.mean(pcd.compute_nearest_neighbor_distance())
nn_distance

0.02887541381235434

In [117]:
radius_normals = nn_distance * 4 
pcd_downsampled.estimate_normals(search_param = o3d.geometry.KDTreeSearchParamHybrid(radius = radius_normals, max_nn = nn), fast_normal_computation=True)

In [108]:
pcd_downsampled.paint_uniform_color([0.6, 0.6, 0.6])
o3d.visualization.draw_geometries([pcd_downsampled, outliers], zoom=zoom, front=front, lookat=lookat, up=up)

In [109]:
pcd = pcd_downsampled
o3d.visualization.draw_geometries([pcd], zoom=zoom, front=front, lookat=lookat, up=up)

In [118]:
o3d.io.write_point_cloud(str(input_path / 'processed_sparse.ply'), pcd)

True

### RANSAC Planar Segmentation

In [78]:
pt_to_plane_dist = 0.02 # how far it should be maximally to be considered inlier 

plane_model, inliers = pcd.segment_plane(distance_threshold=pt_to_plane_dist, ransac_n=3, num_iterations=1000)
[a, b, c, d] = plane_model

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([0,1,0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], zoom=zoom, front=front, lookat=lookat, up=up)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


### Multi-Order RANSAC

In [80]:
max_plane_idx = 6
pt_to_plane_dist = 0.02 

segment_models = {}
segments = {}
rest = pcd

for i in range(max_plane_idx):
    colors = plt.get_cmap('tab20')(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist, ransac_n=3, num_iterations=1000)   
    segments[i] = rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print(f"pass {i} / {max_plane_idx} done.")

pass 0 / 6 done.
pass 1 / 6 done.
pass 2 / 6 done.
pass 3 / 6 done.
pass 4 / 6 done.
pass 5 / 6 done.


In [81]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)] + [rest], 
                                  zoom=zoom, front=front, lookat=lookat, up=up)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


### Euclidean Clustering Refine
Use remaining points to cluster them. 

In [82]:
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()
print(f"point cloud has {max_label + 1} labels")

point cloud has 311 labels


In [83]:
colors = plt.get_cmap('tab20')(labels / min(max_label, 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

In [84]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)] + [rest], 
                                  zoom=zoom, front=front, lookat=lookat, up=up)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
